## 질의응답 모델

질의응답 데이터셋으로 유명한 SQuAD (Stanford Question Answering Dataset) 사용 예정
1.1 버전과 2.0 버전중에서 2.0 버전을 사용하려고 함

In [1]:
import torch
from transformers import BertTokenizer, BertForQuestionAnswering

In [2]:
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [3]:
# input
# [CLS] who was jim henson ? [SEP] jim henson was a nice puppet [SEP]
#   0    1   2   3   4     5   6    7   8      9  10  11   12     13
# 정답의 start_position 은 10, end_position 은 12

input_ids = [[101, 2040, 2001, 3958, 27227, 1029, 102, 3958, 27227, 2001, 1037, 3835, 13997, 102]]
token_type_ids = [[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]
attention_mask = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [4]:
inputs = {
    'input_ids':torch.tensor(input_ids),
    'token_type_ids':torch.tensor(token_type_ids),
    'attention_mask':torch.tensor(attention_mask)
}
start_position = torch.tensor([[10]])
end_position = torch.tensor([[12]])

## SQuAD 데이터셋 이용

위에서 예로 들었던 input 이 SQuAD 데이터셋 형식이다.

In [5]:
import os
import torch
from tqdm import tqdm, trange
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from torch import nn
from torch.utils.data import DataLoader

### 토크나이저 변수 선언

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

### Dataset

In [ ]:
!python -m wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

In [ ]:
!python -m wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

In [ ]:
%mkdir squad

In [ ]:
%mv *-v2.0* squad

In [11]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 unique_id,
                 example_index,
                 doc_span_index,
                 tokens,
                 token_to_orig_map,
                 token_is_max_context,
                 input_ids,
                 input_mask,
                 segment_ids,
                 cls_index,
                 p_mask,
                 paragraph_len,
                 start_position=None,
                 end_position=None,
                 is_impossible=None):
        self.unique_id = unique_id
        self.example_index = example_index
        self.doc_span_index = doc_span_index
        self.tokens = tokens
        self.token_to_orig_map = token_to_orig_map
        self.token_is_max_context = token_is_max_context
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.cls_index = cls_index
        self.p_mask = p_mask
        self.paragraph_len = paragraph_len
        self.start_position = start_position
        self.end_position = end_position
        self.is_impossible = is_impossible
        
def _improve_answer_span(doc_tokens, input_start, input_end, tokenizer,
                         orig_answer_text):
    """Returns tokenized answer spans that better match the annotated answer."""

    # The SQuAD annotations are character based. We first project them to
    # whitespace-tokenized words. But then after WordPiece tokenization, we can
    # often find a "better match". For example:
    #
    #   Question: What year was John Smith born?
    #   Context: The leader was John Smith (1895-1943).
    #   Answer: 1895
    #
    # The original whitespace-tokenized answer will be "(1895-1943).". However
    # after tokenization, our tokens will be "( 1895 - 1943 ) .". So we can match
    # the exact answer, 1895.
    #
    # However, this is not always possible. Consider the following:
    #
    #   Question: What country is the top exporter of electornics?
    #   Context: The Japanese electronics industry is the lagest in the world.
    #   Answer: Japan
    #
    # In this case, the annotator chose "Japan" as a character sub-span of
    # the word "Japanese". Since our WordPiece tokenizer does not split
    # "Japanese", we just use "Japanese" as the annotation. This is fairly rare
    # in SQuAD, but does happen.
    tok_answer_text = " ".join(tokenizer.tokenize(orig_answer_text))

    for new_start in range(input_start, input_end + 1):
        for new_end in range(input_end, new_start - 1, -1):
            text_span = " ".join(doc_tokens[new_start:(new_end + 1)])
            if text_span == tok_answer_text:
                return (new_start, new_end)

    return (input_start, input_end)

def _check_is_max_context(doc_spans, cur_span_index, position):
    """Check if this is the 'max context' doc span for the token."""

    # Because of the sliding window approach taken to scoring documents, a single
    # token can appear in multiple documents. E.g.
    #  Doc: the man went to the store and bought a gallon of milk
    #  Span A: the man went to the
    #  Span B: to the store and bought
    #  Span C: and bought a gallon of
    #  ...
    #
    # Now the word 'bought' will have two scores from spans B and C. We only
    # want to consider the score with "maximum context", which we define as
    # the *minimum* of its left and right context (the *sum* of left and
    # right context will always be the same, of course).
    #
    # In the example the maximum context for 'bought' would be span C since
    # it has 1 left context and 3 right context, while span B has 4 left context
    # and 0 right context.
    best_score = None
    best_span_index = None
    for (span_index, doc_span) in enumerate(doc_spans):
        end = doc_span.start + doc_span.length - 1
        if position < doc_span.start:
            continue
        if position > end:
            continue
        num_left_context = position - doc_span.start
        num_right_context = end - position
        score = min(num_left_context, num_right_context) + 0.01 * doc_span.length
        if best_score is None or score > best_score:
            best_score = score
            best_span_index = span_index

    return cur_span_index == best_span_index

def convert_examples_to_features(examples, tokenizer, max_seq_length,
                                 doc_stride, max_query_length, is_training,
                                 cls_token_at_end=False,
                                 cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                 sequence_a_segment_id=0, sequence_b_segment_id=1,
                                 cls_token_segment_id=0, pad_token_segment_id=0,
                                 mask_padding_with_zero=True):
    """Loads a data file into a list of `InputBatch`s."""

    unique_id = 1000000000
    # cnt_pos, cnt_neg = 0, 0
    # max_N, max_M = 1024, 1024
    # f = np.zeros((max_N, max_M), dtype=np.float32)

    do_print = False
    features = []
    for (example_index, example) in enumerate(trange(len(examples))):

        # if example_index % 100 == 0:
        #     logger.info('Converting %s/%s pos %s neg %s', example_index, len(examples), cnt_pos, cnt_neg)

        example = examples[example_index]
        query_tokens = tokenizer.tokenize(example.question)

        # 쿼리 길이가 길 경우 max_query_length로 자르기
        if len(query_tokens) > max_query_length:
            query_tokens = query_tokens[0:max_query_length]

        # example.context: [The, Normans, ...]
        # token: The, Normans, ...
        # subtoken: the, norman, ##s
        # tok_to_orig_index: 0, 1, 1, ...
        # orig_to_tok_index: 0, 2, ...
        tok_to_orig_index = []
        orig_to_tok_index = []
        all_doc_tokens = []
        for (i, token) in enumerate(example.context):
            orig_to_tok_index.append(len(all_doc_tokens))
            sub_tokens = tokenizer.tokenize(token)
            for sub_token in sub_tokens:
                tok_to_orig_index.append(i)
                all_doc_tokens.append(sub_token)

        tok_start_position = None
        tok_end_position = None
        if is_training and example.is_impossible:
            tok_start_position = -1
            tok_end_position = -1
        if is_training and not example.is_impossible:
            tok_start_position = orig_to_tok_index[example.start]
            if example.end < len(example.context) - 1:
                tok_end_position = orig_to_tok_index[example.end + 1] - 1
            else:
                tok_end_position = len(all_doc_tokens) - 1
            (tok_start_position, tok_end_position) = _improve_answer_span(
                all_doc_tokens, tok_start_position, tok_end_position, tokenizer,
                example.answer)

        # The -3 accounts for [CLS], [SEP] and [SEP]
        max_tokens_for_doc = max_seq_length - len(query_tokens) - 3

        # We can have documents that are longer than the maximum sequence length.
        # To deal with this we do a sliding window approach, where we take chunks
        # of the up to our max length with a stride of `doc_stride`.
        _DocSpan = collections.namedtuple(  # pylint: disable=invalid-name
            "DocSpan", ["start", "length"])
        doc_spans = []
        start_offset = 0
        while start_offset < len(all_doc_tokens):
            length = len(all_doc_tokens) - start_offset
            if length > max_tokens_for_doc:
                length = max_tokens_for_doc
            doc_spans.append(_DocSpan(start=start_offset, length=length))
            if start_offset + length == len(all_doc_tokens):
                break
            start_offset += min(length, doc_stride)

        if len(doc_spans) == 2:
            do_print = True

        for (doc_span_index, doc_span) in enumerate(doc_spans):
            tokens = []
            token_to_orig_map = {}
            token_is_max_context = {}
            segment_ids = []

            # p_mask: mask with 1 for token than cannot be in the answer (0 for token which can be in an answer)
            # Original TF implem also keep the classification token (set to 0) (not sure why...)
            p_mask = []

            # CLS token at the beginning
            if not cls_token_at_end:
                tokens.append(cls_token)
                segment_ids.append(cls_token_segment_id)
                p_mask.append(0)
                cls_index = 0

            # Query
            for token in query_tokens:
                tokens.append(token)
                segment_ids.append(sequence_a_segment_id)
                p_mask.append(1)

            # SEP token
            tokens.append(sep_token)
            segment_ids.append(sequence_a_segment_id)
            p_mask.append(1)

            # Paragraph
            for i in range(doc_span.length):
                split_token_index = doc_span.start + i
                token_to_orig_map[len(tokens)] = tok_to_orig_index[split_token_index]

                is_max_context = _check_is_max_context(doc_spans, doc_span_index,
                                                       split_token_index)
                token_is_max_context[len(tokens)] = is_max_context
                tokens.append(all_doc_tokens[split_token_index])
                segment_ids.append(sequence_b_segment_id)
                p_mask.append(0)
            paragraph_len = doc_span.length

            # SEP token
            tokens.append(sep_token)
            segment_ids.append(sequence_b_segment_id)
            p_mask.append(1)

            # CLS token at the end
            if cls_token_at_end:
                tokens.append(cls_token)
                segment_ids.append(cls_token_segment_id)
                p_mask.append(0)
                cls_index = len(tokens) - 1  # Index of classification token

            input_ids = tokenizer.convert_tokens_to_ids(tokens)

            # The mask has 1 for real tokens and 0 for padding tokens. Only real
            # tokens are attended to.
            input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

            # Zero-pad up to the sequence length.
            while len(input_ids) < max_seq_length:
                input_ids.append(pad_token)
                input_mask.append(0 if mask_padding_with_zero else 1)
                segment_ids.append(pad_token_segment_id)
                p_mask.append(1)

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            # start_position의 위치가 doc_span의 길이 전체를 넘어버릴 경우 is_impossible=True로 수정하기
            span_is_impossible = example.is_impossible
            start_position = None
            end_position = None
            if is_training and not span_is_impossible:
                # For training, if our document chunk does not contain an annotation
                # we throw it out, since there is nothing to predict.
                doc_start = doc_span.start
                doc_end = doc_span.start + doc_span.length - 1
                out_of_span = False
                if not (tok_start_position >= doc_start and
                        tok_end_position <= doc_end):
                    out_of_span = True
                if out_of_span:
                    start_position = 0
                    end_position = 0
                    span_is_impossible = True
                else:
                    doc_offset = len(query_tokens) + 2
                    start_position = tok_start_position - doc_start + doc_offset
                    end_position = tok_end_position - doc_start + doc_offset

            if is_training and span_is_impossible:
                start_position = cls_index
                end_position = cls_index

            features.append(
                InputFeatures(
                    unique_id=unique_id,
                    example_index=example_index,
                    doc_span_index=doc_span_index,
                    tokens=tokens,
                    token_to_orig_map=token_to_orig_map,
                    token_is_max_context=token_is_max_context,
                    input_ids=input_ids,
                    input_mask=input_mask,
                    segment_ids=segment_ids,
                    cls_index=cls_index,
                    p_mask=p_mask,
                    paragraph_len=paragraph_len,
                    start_position=start_position,
                    end_position=end_position,
                    is_impossible=span_is_impossible))
            unique_id += 1

    return features

In [45]:
import os
import json
from torch.utils.data import Dataset
import collections

def is_whitespace(c):
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False

class SquadExample():
    def __init__(self, qid, context, question, answer, start, end, is_impossible):
        self.qid = qid
        self.context = context
        self.question = question
        self.answer = answer
        self.start = start
        self.end = end
        self.is_impossible = is_impossible
        
    def __repr__(self):
        #return self.context[self.start:self.end]
        #if self.context[self.start:self.end] != self.answer:
        #    return 'NA!! {} - {}'.format(self.context[self.start:self.end], answer)
        return 'id:{}  question:{}...  answer:{}...  is_impossible:{}'.format(
            self.qid,
            self.question[:10],
            self.answer[:10],
            self.is_impossible)

class SquadDataset(Dataset):
    def __init__(self, path, tokenizer, is_train=True, is_inference=False):
        '''
        path: SquadDataset 데이터셋 위치
        tokenizer: Squad 데이터셋을 토크나이징할 토크나이저, ex) BertTokenizer
        is_train: SquadDataset을 정의하는 목적이 모델 학습용일 경우 True, 그렇지 않으면 False
        is_inference: SquadDataset을 정의하는 목적이 인퍼런스용일 경우 True, 그렇지 않으면 False
        '''
        
        if is_train:
            filename = os.path.join(path, 'train-v2.0.json')
        else:
            if is_inference:
                filename = os.path.join(path, 'test-v2.0.json')
            else:
                filename = os.path.join(path, 'dev-v2.0.json')

        cached_features_file = os.path.join(os.path.dirname(filename), 'cached_{}_64.cache'.format('train' if is_train else 'valid'))
        #cached_examples_file = os.path.join(os.path.dirname(filename), 'cached_example_{}_64.cache'.format('train' if is_train else 'valid'))

        if os.path.exists(cached_features_file):
            print('cache file exists')
            #self.examples = torch.load(cached_examples_file)
            self.features = torch.load(cached_features_file)
        else:
            print('cache file does not exist')

            with open(filename, "r", encoding='utf-8') as reader:
                input_data = json.load(reader)["data"]
                
            example_count = 0

            self.examples = []
            for entry in input_data:
                for paragraph in entry["paragraphs"]:
                    context = paragraph['context']
                    
                    doc_tokens = []
                    char_to_word_offset = []
                    prev_is_whitespace = True
                    for c in context:
                        if is_whitespace(c):
                            prev_is_whitespace = True
                        else:
                            if prev_is_whitespace:
                                doc_tokens.append(c)
                            else:
                                doc_tokens[-1] += c
                            prev_is_whitespace = False
                        char_to_word_offset.append(len(doc_tokens) - 1)
                            
                            
                    for qa in paragraph['qas']:
                        is_impossible = qa['is_impossible']
                        
                        if not is_impossible:
                            answer = qa['answers'][0]
                            original_answer = answer['text']
                            answer_start = answer['answer_start']
                            
                            answer_length = len(original_answer)
                            start_pos = char_to_word_offset[answer_start]
                            end_pos = char_to_word_offset[answer_start + answer_length - 1]

                            answer_end = answer_start + len(original_answer)
                        else:
                            original_answer = ''
                            start_pos = 1
                            end_pos = -1

                        example = SquadExample(
                            qid=qa['id'],
                            context=doc_tokens,
                            question=qa['question'],
                            answer=original_answer,
                            start=start_pos,
                            end=end_pos,
                            is_impossible=is_impossible)
                        self.examples.append(example)
                        
                        example_count += 1
                        if example_count >= 500:
                            break
                    if example_count >= 500:
                            break
                if example_count >= 500:
                            break
            print('examples: {}'.format(len(self.examples)))

            self.features = convert_examples_to_features(
                examples=self.examples,
                tokenizer=tokenizer,
                max_seq_length=384,
                doc_stride=128,
                max_query_length=64,
                is_training=True if not is_inference else False)
            print('is_training: {}'.format(True if not is_inference else False))

            # torch.save(self.examples, cached_examples_file)
            torch.save(self.features, cached_features_file)

        '''
        # Convert to Tensors and build dataset
        all_input_ids = torch.tensor([f.input_ids for f in self.features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in self.features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in self.features], dtype=torch.long)
        all_cls_index = torch.tensor([f.cls_index for f in self.features], dtype=torch.long)
        all_p_mask = torch.tensor([f.p_mask for f in self.features], dtype=torch.float)
        if is_train:
            all_start_positions = torch.tensor([f.start_position for f in self.features], dtype=torch.long)
            all_end_positions = torch.tensor([f.end_position for f in self.features], dtype=torch.long)
            dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                                    all_start_positions, all_end_positions,
                                    all_cls_index, all_p_mask)
        else:
            all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
            dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_example_index, all_cls_index, all_p_mask)
        return dataset
        '''


    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx]

In [47]:
%rm -rf squad/*.cache

In [48]:
train_dataset = SquadDataset('squad', tokenizer, is_train=True)

cache file does not exist
examples: 500


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:02<00:00, 219.47it/s]


is_training: True


### DataLoader

In [49]:
class SquadDataLoader(DataLoader):
    def __init__(self, dataset, batch_size, is_inference=False, shuffle=True):
        '''
        dataset: SquadDataset으로 정의한 데이터셋 객체
        batch_size: 배치 사이즈
        is_inference: SquadDataLoader를 인퍼런스 목적으로 사용할 경우 True, 그렇지 않으면 False
        shuffle: 데이터의 순서를 섞을 경우 True, 그렇지 않으면 False
        '''
        self.is_inference = is_inference
        super().__init__(dataset, collate_fn=self.squad_collate_fn, batch_size=batch_size, shuffle=shuffle)
        
    def squad_collate_fn(self, features):
        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
        all_cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
        all_p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)

        # return 6 tensors
        if self.is_inference:
            all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
            return all_input_ids, all_input_mask, all_segment_ids, all_cls_index, all_p_mask, all_example_index
        # return 7 tensors
        else:
            all_start_positions = torch.tensor([f.start_position for f in features], dtype=torch.long)
            all_end_positions = torch.tensor([f.end_position for f in features], dtype=torch.long)
            return all_input_ids, all_input_mask, all_segment_ids, all_cls_index, all_p_mask, all_start_positions, all_end_positions

In [50]:
#train_dataloader = SquadDataLoader(train_dataset, batch_size=16, is_inference=False, shuffle=True)
train_dataloader = SquadDataLoader(train_dataset, batch_size=4, is_inference=False, shuffle=True)

In [51]:
for i, batch in enumerate(train_dataloader):
    if i > 10:
        break
    input_ids, input_mask, segment_ids, cls_index, p_mask, start_positions, end_positions = batch
    print(input_ids.shape, input_mask.shape, segment_ids.shape, cls_index.shape, p_mask.shape, start_positions.shape, end_positions.shape)


torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4]) torch.Size([4, 384]) torch.Size([4]) torch.Size([4])
torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4]) torch.Size([4, 384]) torch.Size([4]) torch.Size([4])
torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4]) torch.Size([4, 384]) torch.Size([4]) torch.Size([4])
torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4]) torch.Size([4, 384]) torch.Size([4]) torch.Size([4])
torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4]) torch.Size([4, 384]) torch.Size([4]) torch.Size([4])
torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4]) torch.Size([4, 384]) torch.Size([4]) torch.Size([4])
torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4]) torch.Size([4, 384]) torch.Size([4]) torch.Size([4])
torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4, 384]) torch.Size([4

### Train

In [52]:
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [ ]:
model.to("mps")

In [53]:
model.train()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [54]:
# Optimizer와 Loss 함수는 가장 일반적인 것으로 정의했다.
# 이 노트북 파일의 목적은 BERT를 이용해서 높은 성능의 모델을 간편하게 만들 수 있다는 것을 보여주기 위함이다.
# Optimizer와 Loss를 최적화할 경우 좋은 성능이 나온 이유를 잘 설명할 수 없다.
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
loss = nn.CrossEntropyLoss()

In [55]:
def train(model, dataloader, optimizer):
    tbar = tqdm(dataloader, desc='Training', leave=True)
    
    total_loss = 0.0
    for i, batch in enumerate(tbar):
        optimizer.zero_grad()
        
        # cls_index와 p_mask는 XLNet 모델에 사용되므로 BERT에서는 사용하지 않는다.
        input_ids, input_mask, segment_ids, cls_index, p_mask, start_positions, end_positions = batch
        
        # to cuda
        if torch.cuda.is_available():
            input_ids = input_ids.cuda()
            input_mask = input_mask.cuda()
            segment_ids = segment_ids.cuda()
            start_positions = start_positions.cuda()
            end_positions = end_positions.cuda()
            
        #if torch.backends.mps.is_available():
        #    input_ids = input_ids.to("mps")
        
        # train model
        #out = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        inputs = {
            'input_ids': input_ids,
            'token_type_ids': segment_ids,
            'attention_mask': input_mask,
        }
        out = model(**inputs, start_positions=start_positions, end_positions=end_positions)
        loss = out.loss

        loss.backward()
        optimizer.step()
        
        total_loss += loss.data.item()
        tbar.set_description("Average Loss = {:.4f})".format(total_loss/(i+1)))

In [56]:
n_epoch = 10

In [57]:
for i in range(n_epoch):
    train(model, train_dataloader, optimizer)

Average Loss = 0.1499): 100%|█████████████████████████████████████████████████████████| 132/132 [04:09<00:00,  1.89s/it]


In [58]:
torch.save(model.state_dict(), 'squad_model.bin')

### Inference

학습한 모델을 로딩해서 Inference 하는 코드

In [59]:
from transformers import BertConfig

#model2 = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2)
bert_config = BertConfig.from_pretrained("bert-base-uncased", num_labels = 2)
model2 = BertForQuestionAnswering(bert_config)
                                       
# 학습한 모델 로딩
model2.load_state_dict(torch.load('./squad_model.bin', map_location='cpu'))
#model.load_state_dict(torch.load('cola_model_no_pretrained.bin', map_location='cpu'))
model2.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [68]:
!python run_evaluate.py --cache_dir=squad --version_2_with_negative

08/13/2022 22:37:37 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSe

08/13/2022 22:37:45 - INFO - utils_squad -   *** Example ***
08/13/2022 22:37:45 - INFO - utils_squad -   unique_id: 1000000006
08/13/2022 22:37:45 - INFO - utils_squad -   example_index: 6
08/13/2022 22:37:45 - INFO - utils_squad -   doc_span_index: 0
08/13/2022 22:37:45 - INFO - utils_squad -   tokens: [CLS] what is france a region of ? [SEP] the norman ##s ( norman : no ##ur ##man ##ds ; french : norman ##ds ; latin : norman ##ni ) were the people who in the 10th and 11th centuries gave their name to normandy , a region in france . they were descended from norse ( " norman " comes from " norse ##man " ) raiders and pirates from denmark , iceland and norway who , under their leader roll ##o , agreed to swear fe ##al ##ty to king charles iii of west fran ##cia . through generations of assimilation and mixing with the native frankish and roman - gaul ##ish populations , their descendants would gradually merge with the carol ##ing ##ian - based cultures of west fran ##cia . the distinct

08/13/2022 22:37:45 - INFO - utils_squad -   *** Example ***
08/13/2022 22:37:45 - INFO - utils_squad -   unique_id: 1000000013
08/13/2022 22:37:45 - INFO - utils_squad -   example_index: 13
08/13/2022 22:37:45 - INFO - utils_squad -   doc_span_index: 0
08/13/2022 22:37:45 - INFO - utils_squad -   tokens: [CLS] who was famed for their christian spirit ? [SEP] the norman dynasty had a major political , cultural and military impact on medieval europe and even the near east . the norman ##s were famed for their martial spirit and eventually for their christian pie ##ty , becoming expo ##nent ##s of the catholic orthodoxy into which they ass ##imi ##lated . they adopted the gallo - romance language of the frankish land they settled , their dialect becoming known as norman , norma ##und or norman french , an important literary language . the duchy of normandy , which they formed by treaty with the french crown , was a great fi ##ef of medieval france , and under richard i of normandy was fo

convertting examples to features: 100%|██| 11873/11873 [00:47<00:00, 250.28it/s]
08/13/2022 22:38:33 - INFO - __main__ -   Saving features into cached file squad/cached_dev_bert-base-uncased_384_features
08/13/2022 22:38:39 - INFO - __main__ -   ***** Running evaluation  *****
08/13/2022 22:38:39 - INFO - __main__ -     Num examples = 12232
08/13/2022 22:38:39 - INFO - __main__ -     Batch size = 8
Evaluating: 100%|███████████████████████████| 1529/1529 [24:46<00:00,  1.03it/s]
08/13/2022 23:03:26 - INFO - utils_squad -   Writing predictions to: outputs/predictions_.json
08/13/2022 23:03:26 - INFO - utils_squad -   Writing nbest to: outputs/nbest_predictions_.json
examples: 11873
features: 12232
all_results: 12232


In [71]:
!python evaluate.py squad/dev-v2.0.json outputs/predictions_.json

{
  "exact": 23.01861366124821,
  "f1": 29.224821740750045,
  "total": 11873,
  "HasAns_exact": 45.49595141700405,
  "HasAns_f1": 57.92616540619522,
  "HasAns_total": 5928,
  "NoAns_exact": 0.6055508830950378,
  "NoAns_f1": 0.6055508830950378,
  "NoAns_total": 5945
}
